# Connecting to NYT API

## Libraries

In [1]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

In [75]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

## Calling the API

In [124]:
key = 'kAD5GYrJCPoBK6HAdLQH3TTSFiQ3fRCO'

url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=("election" "trump" "clinton")&facet_field=day_of_week&facet=true&begin_date=20161001&end_date=20161001&api-key='+key

response = requests.get(url)
results = response.json()

results

#need to further refine filtering to only incluse US election (maybe politics section)
#loop by day

{'status': 'OK',
 'copyright': 'Copyright (c) 2019 The New York Times Company. All Rights Reserved.',
 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2016/10/01/us/politics/donald-trump-interview-bill-hillary-clinton.html',
    'snippet': 'In an interview, Mr. Trump said he was reconsidering whether he would back Mrs. Clinton if she became president.',
    'lead_paragraph': 'Donald J. Trump unleashed a slashing new attack on Hillary Clinton over Bill Clinton’s sexual indiscretions on Friday as he sought to put the Clintons’ relationship at the center of his political argument against her before their next debate.',
    'abstract': 'In an interview, Mr. Trump said he was reconsidering whether he would back Mrs. Clinton if she became president.',
    'print_page': '10',
    'source': 'The New York Times',
    'multimedia': [{'rank': 0,
      'subtype': 'thumbnail',
      'caption': None,
      'credit': None,
      'type': 'image',
      'url': 'images/2016/10/01/us/01TRUMPIN

## Exploring the JSON for the Headline

In [62]:
results['response']['docs'][0].keys()

dict_keys(['web_url', 'snippet', 'lead_paragraph', 'abstract', 'print_page', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'subsection_name', 'byline', 'type_of_material', '_id', 'word_count', 'uri'])

#### Publication Date

In [137]:
results['response']['docs'][0]['pub_date'][:10]

'2016-10-01'

#### Headline

In [64]:
results['response']['docs'][0]['headline']['main']

'Donald Trump Opens New Line of Attack on Hillary Clinton: Her Marriage'

In [65]:
results['response']['docs'][1]['headline']['main']

'Hillary Clinton and Democrats Raised $154 Million in September'

In [101]:
results['response']['docs'][8]['headline']['main']

'The Monster Sorority of Women Voters'

In [99]:
len(results['response']['docs'])

9

In [36]:
#seems like there's no need to normalize the Json

#articles = json_normalize(results)

#articles

#articles['response.docs'][0]

## Loop to gather multiple titles

In [125]:
quant = 1

titles = []

for i in range(quant):
    headline = results['response']['docs'][i]['headline']['main']
    titles.append(headline)

In [71]:
titles

['Donald Trump Opens New Line of Attack on Hillary Clinton: Her Marriage',
 'Hillary Clinton and Democrats Raised $154 Million in September',
 'New Hillary Clinton Ads Look to Expand Lead With Hispanic Voters',
 'How Could Anyone Vote for Trump?',
 'Donald Trump Is Seen as Helping Push Asian-Americans Into Democratic Arms']

## Loop to iterate through multiple days

#### And save it a dataset

In [139]:
days = [str(i) for i in range(1,31)]       
titles = []


for i in days:
    url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=("election" "trump" "clinton")&facet_field=day_of_week&facet=true&begin_date=201610'+i+'&end_date=201610'+i+'&api-key='+key
    response = requests.get(url)
    results = response.json()
    
    try:
        quant = len(results['response']['docs'])
    
        print (i)

        for y in range(quant):
            pubdate = results['response']['docs'][0]['pub_date'][:10]
            headline = results['response']['docs'][y]['headline']['main']
            article_dict = {'Date':pubdate, 'Headline': headline}
            titles.append(article_dict)
        
    except KeyError:
        pass
            


1
2
3
4
5
6
7
8
9
10
11
12
13
16
17
18
19
20
21
22
23
24
25


In [140]:
titles

[{'Date': '2016-10-01',
  'Headline': 'Donald Trump Opens New Line of Attack on Hillary Clinton: Her Marriage'},
 {'Date': '2016-10-01',
  'Headline': 'Hillary Clinton and Democrats Raised $154 Million in September'},
 {'Date': '2016-10-01',
  'Headline': 'New Hillary Clinton Ads Look to Expand Lead With Hispanic Voters'},
 {'Date': '2016-10-01', 'Headline': 'How Could Anyone Vote for Trump?'},
 {'Date': '2016-10-01',
  'Headline': 'Donald Trump Is Seen as Helping Push Asian-Americans Into Democratic Arms'},
 {'Date': '2016-10-01', 'Headline': 'Donald Trump’s Pathetic Fraternity'},
 {'Date': '2016-10-01', 'Headline': 'On the Trail: Week of Sept. 25'},
 {'Date': '2016-10-01', 'Headline': 'Trump and the Intellectuals'},
 {'Date': '2016-10-01', 'Headline': 'The Monster Sorority of Women Voters'},
 {'Date': '2016-10-02',
  'Headline': '‘S.N.L.’ Begins a New Season With Alec Baldwin as Donald Trump'},
 {'Date': '2016-10-02',
  'Headline': 'Donald Trump, Hillary Clinton, Hoboken: Your Weeken

In [146]:
df_titles = pd.DataFrame(titles)
df_titles.head()

,Date,Headline
0,2016-10-01,Donald Trump Opens New Line of Attack on Hilla...
1,2016-10-01,Hillary Clinton and Democrats Raised $154 Mill...
2,2016-10-01,New Hillary Clinton Ads Look to Expand Lead Wi...
3,2016-10-01,How Could Anyone Vote for Trump?
4,2016-10-01,Donald Trump Is Seen as Helping Push Asian-Ame...


## Export dataset

In [147]:
df_titles.to_csv('data/nyt.csv', index=False)